In [1]:
from dialogue2graph.pipelines.model_storage import ModelStorage
from dialogue2graph.pipelines.d2g_llm.pipeline import Pipeline as D2GLLMPipeline

/home/askatasuna/Документы/DeepPavlov/chatsky-llm-autoconfig/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(
INFO:datasets:PyTorch version 2.6.0 available.


In [2]:
ms = ModelStorage()

pipe = D2GLLMPipeline(ms)

/home/askatasuna/Документы/DeepPavlov/chatsky-llm-autoconfig/dialogue2graph/pipelines/model_storage.py:60: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(**item.config)
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3


In [ ]:
pipe.invoke()